In [3]:
from IPython.display import display, Markdown

import pandas as pd
import gc
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold, cross_val_score

from sklearn.metrics import roc_auc_score

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

from sklearn.tree import plot_tree

from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler, OneHotEncoder

from scipy.sparse import hstack

# XGBoost
import xgboost as xgb

import numpy as np

import matplotlib.pyplot as plt

import itertools

import pandas as pd # Para cargar los datos y hacer OHE.
import numpy as np  # Para lidiar con NaNs.
import time
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import confusion_matrix

random_state = 42
np.random.seed(random_state)


In [9]:
from scipy.stats import uniform
params = {'max_depth': list(range(1, 20)),
          'learning_rate': uniform(scale = 0.15),
          'gamma': uniform(scale = 2),
          'reg_lambda': uniform(scale = 5),        # Parámetro de regularización.
          'subsample': uniform(0.5, 0.5),          # Entre 0.5 y 1.
          'min_child_weight': uniform(scale = 5),
          'colsample_bytree': uniform(0.75, 0.25), # Entre 0.75 y 1.
          'n_estimators': list(range(1, 300))
         }

# Load the competition data
comp_data = pd.read_csv("./competition_data.csv")


# Split into training and evaluation samples
train_data = comp_data[comp_data["ROW_ID"].isna()]
eval_data = comp_data[comp_data["ROW_ID"].notna()]
del comp_data
gc.collect()

y_train = train_data["conversion"]
X_train = train_data.drop(columns=["conversion", "ROW_ID"])
X_train = X_train.select_dtypes(include='number') ## retoque de datos para que haya mas
del train_data
gc.collect()

#hold out set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=3456)

# # Train a random forest model on the train data
# train_data = train_data.sample(frac=1/3)
# y_train = train_data["conversion"]
# X_train = train_data.drop(columns=["conversion", "ROW_ID"])
# X_train = X_train.select_dtypes(include='number')
# del train_data
# gc.collect()
start = time.time()–
best_score = 0
best_estimator = None
iterations = 1
y_preds_list = []
for g in ParameterSampler(params, n_iter = iterations, random_state = 2345):
    cls = xgb.XGBClassifier(objective = 'binary:logistic', seed = 2345, eval_metric = 'auc', **g)
    # cls.fit(X_train + X_val, y_train)
    cls.fit(X_train, y_train)
    
    # Predict on the evaluation set
    # y_preds = cls.predict_proba(eval_data.drop(columns=["ROW_ID"]))[:, cls.classes_ == 1].squeeze()
    y_preds = cls.predict_proba(X_val)[:, 1] 
    y_preds_list.append(y_preds)

    print(roc_auc_score(y_val,y_preds))
    print(g)
    y_preds = None




0.7907696632911395
{'colsample_bytree': 0.9151610773058515, 'gamma': 0.827201304042807, 'learning_rate': 0.10021538369719958, 'max_depth': 14, 'min_child_weight': 3.4391200843333576, 'n_estimators': 123, 'reg_lambda': 3.184092322548138, 'subsample': 0.58609692603256}


In [10]:
np.shape(y_preds_list[0])


(36153,)

In [6]:
# y_preds = cls.predict_proba(eval_data.drop(columns=["ROW_ID"]))[:, cls.classes_ == 1].squeeze()
# Make the submission file
submission_df = pd.DataFrame({"ROW_ID": eval_data["ROW_ID"], "conversion": y_preds})
submission_df["ROW_ID"] = submission_df["ROW_ID"].astype(int)
submission_df.to_csv("basic_model_xgboostoro.csv", sep=",", index=False)

ValueError: array length 36153 does not match index length 19211